# Resonator Design

### Quartz equivalent circuit
<div>
<img src="images/quartz_eq_circuit.png" width="800"/>
</div>

There is a small region where the impedance of the quartz / resonator is such that it oscillates.
<div>1000
<img src="images/resonator_oscillation_impedance.png" width="600"/>
</div>

By varying the impedance of the crystal we can change its oscillation frequency.

### Load Capacitance
Determined by external capacitors and stray board capacitance. The oscillator must show the same load capacitance as the one the crystal was adjusted for.
<div>
<img src="images/resonator_load_capacitance.png" width="200"/>
</div>

* $C_{L}$: Load capacitance required
* $C_{L1/2}$: Externally applied capacitors
* $C_{s}$: Stray board capacitance (PCB + MCU pin capacitance: assumed 10 pF according to stm32-datasheet)

In [10]:
import math
### Example calculation: ABLS-8.000MHZ-20-B-3-H-T
C_S = 10e-12 # PCB datasheet
C_L = 18e-12 # Input load capacitance (resonator datasheet)
# Assume C_L1 = C_L2
C_L1 = (C_L - C_S)*2
C_L2 = C_L1
print(f"Load capacitances: Cl1 {C_L1*1e12}pF, Cl2 {C_L2*1e12}pF")

Load capacitances: Cl1 16.0pF, Cl2 16.0pF


## Resonator compatibility

### PCB oscillator stability
One should make sure that the PCB's oscillator driver is actually strong enough to driver the resonator into stable resonance:
* The oscillation loop critical transconductance ($Gm_{crit\_max}$) should be greater than the oscillation loop critical gain. ($Gm_{crit}$)
* The gain margin ratio $ GM = \frac{g_m}{Gm_{crit}}$ > 5, with $g_m$ the transconductance.

In [6]:
import math
### Example calculation: ABLS-8.000MHZ-20-B-3-H-T
C_0 = 7e-12 # Shunt capacitance (resonator datasheet)
C_L = 18e-12 # Input load capacitance (resonator datasheet)
ESR = 80 # (resonator datasheet)
F_OSCILLATOR = 8e6
trancond = 10e-3 # in A / V, oscillator transconductance (check stm32 datasheet for it)
# Assume C_L1 = C_L2
g_m_crit = 4 * ESR * (2 * math.pi * F_OSCILLATOR) ** 2 * (C_0 + C_L) **2
GM = trancond / g_m_crit
print(f"Gain margin: ({GM} > 5) ? | g_m_crit: {g_m_crit}")

Gain margin: (19.7892936801441 > 5) ? | g_m_crit: 0.000505323745335775


### PCB oscillator safety

One should make sure the PCB's oscillator driver doesn't break the resonator by driving it too hard. Therefor we should check
* The drive level: $DL = I^{2}_{Crystal, RMS}*ESR = ESR * (\pi * F * C_{tot})²*(V_{pp})² / 2$, which may not exceed the DL specified by the manufacturer.
* The measurements however need to be done by hand, measuring the P2P voltage swing with a probe.

<div>
<img src="images/resonator_DL.png" width="600"/>
</div>

In [ ]:
import math
### Example calculation: ABLS-8.000MHZ-20-B-3-H-T
ESR = 80 # resonator resistance
F_OSCILLATOR = 8e6 
C_PROBE = 0e-12
C_S = 10e-12 # PCB datasheet
C_TOT = C_L1 + (C_S/2) + C_PROBE
V_PP =  0 # Is the peak2peak voltage level measured at the C_L1

To be sure we will simply check whether the drive level is good enough, and put a 0-ohm resistor in series.

**Sources:**
> https://www.st.com/resource/en/datasheet/stm32f303cb.pdf \
> https://www.st.com/resource/en/application_note/an2867-guidelines-for-oscillator-design-on-stm8afals-and-stm32-mcusmpus-stmicroelectronics.pdf \
> https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/lcsc/2304140030_Abracon-LLC-ABLS-8-000MHZ-20-B-3-H-T_C1987070.pdf

### Example calculation 1: 7325-0800A2010-00
https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/lcsc/1912111437_XTY-7325-0800A2010-00_C389817.pdf

In [5]:
import math
### Example calculation: ABLS-8.000MHZ-20-B-3-H-T
C_S = 10e-12 # PCB datasheet
C_L = 20e-12 # Input load capacitance (resonator datasheet)
C_0 = 4e-12 # Shunt capacitance (resonator datasheet)
ESR = 200 # (resonator datasheet)
F_OSCILLATOR = 8e6
trancond = 10e-3 # in A / V, oscillator transconductance (check stm32 datasheet for it)


In [6]:
# Assume C_L1 = C_L2
C_L1 = (C_L - C_S)*2
C_L2 = C_L1
print(f"Load capacitances: Cl1 {C_L1*1e12}pF, Cl2 {C_L2*1e12}pF")

Load capacitances: Cl1 20.0pF, Cl2 20.0pF


In [7]:
import math
# Assume C_L1 = C_L2
g_m_crit = 4 * ESR * (2 * math.pi * F_OSCILLATOR) ** 2 * (C_0 + C_L) **2
GM = trancond / g_m_crit
print(f"Gain margin: ({GM} > 5) ? | g_m_crit: {g_m_crit}")

Gain margin: (8.589103159784766 > 5) ? | g_m_crit: 0.0011642659092536256
